<a href="https://colab.research.google.com/github/Smriti79/Celebal-Technologies/blob/main/AssignmentEight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers faiss-cpu datasets


In [ ]:
from google.colab import files
import os

uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Read file content
with open(filename, 'r', encoding='utf-8') as f:
    document = f.read()

print("Sample document content:\n", document[:500])


Saving Training Dataset.csv to Training Dataset (2).csv
Sample document content:
 Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
LP001002,Male,No,0,Graduate,No,5849,0,,360,1,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358,120,360,1,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0,141,360,1,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196,267,36


In [ ]:
import re

def split_into_chunks(text, max_words=150):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks, current_chunk = [], []

    for sentence in sentences:
        current_chunk.append(sentence)
        if len(' '.join(current_chunk).split()) > max_words:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

chunks = split_into_chunks(document)
print(f"Total chunks created: {len(chunks)}")


Total chunks created: 1


In [ ]:

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub.utils._auth")

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")  # lightweight model
embeddings = embedder.encode(chunks, show_progress_bar=True)

# Index with FAISS
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def retrieve_relevant_chunks(query, k=3):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [chunks[i] for i in indices[0]]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def generate_answer(query):
    relevant_chunks = retrieve_relevant_chunks(query)
    context = " ".join(relevant_chunks)

    prompt = f"Context: {context} \n\n Question: {query} \n Answer:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    outputs = model.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [ ]:
query = "What is the purpose of the document?"
print("User Query:", query)
print("Answer:", generate_answer(query))


User Query: What is the purpose of the document?
Answer: LP001028, Male, Yes, 2, Graduate, 3200, 700, 70, 360, 1, Urban, N LP001027, Male, Yes, 2, Graduate, 3200, 700, 70, 360, 1, Urban, N LP001028, Male, Yes, 2, Graduate, 3200, 700, 70, 360, 1, Urban, N LP001028, Male, Yes, 2, Graduate, 3200, 700, 70, 360,
